In [1]:
import sys
import csv
import dbfread
import shapefile
import pandas as pd

In [2]:
# Lit le fichier .dbf et stocke les données dans un DataFrame
def read_dbf(path_dbf):
    table = dbfread.DBF(path_dbf, encoding='latin-1') # latin-1 pour fix si jamais...
    df = pd.DataFrame(iter(table))
    return df

In [11]:
roads = read_dbf("Sarthe/roads.dbf")
roads.__len__()

13546

In [4]:
points = read_dbf("departement-72/points.dbf")
points.__len__()

30607

In [12]:
# Affiche les différents types de routes
roads["type"].unique()

array(['primary', 'primary_link', 'secondary', 'pedestrian', 'trunk_link',
       'tertiary', 'residential', 'service', 'steps', 'path', 'cycleway',
       'track', 'trunk', 'unclassified', 'footway', 'living_street',
       'secondary_link', 'tertiary_link', 'busway', 'platform',
       'elevator', 'corridor', 'construction', 'proposed'], dtype=object)

In [13]:
# Conserve uniquement les routes de type "motorway, motorway_link, primary, secondary, tertiary, primary_link, secondary_link, tertiary_link, road"
roads = roads[roads["type"].isin(["motorway", "motorway_link", "primary", "secondary", "tertiary", "primary_link", "secondary_link", "tertiary_link", "road"])]
roads.__len__()

1524

In [14]:
# Filtrer le fichier .shp pour ne garder que les routes de type spécifié
def filter_shp(path_shp, path_dbf, path_out, type_route):
    # Lit le fichier .dbf et stocke les données dans un DataFrame
    df = read_dbf(path_dbf)
    # Lit le fichier .shp
    sf = shapefile.Reader(path_shp)
    # Crée un nouveau fichier .shp
    w = shapefile.Writer(path_out, shapefile.POLYLINE)
    # Copie les attributs du fichier .shp
    w.fields = list(sf.fields)
    # Pour chaque route
    for shape in sf.iterShapeRecords():
        # Si la route est du type spécifié
        if shape.record["type"] == type_route:
            # Copie la route dans le nouveau fichier .shp
            w.record(*shape.record)
            w.shape(shape.shape)
    w.close()

In [23]:
# Filtre le fichier roads.shp pour chaque type de route
# Enregistre le résultat dans le fichier routes/roads_<type>.shp
for type_route in roads["type"].unique():
    filter_shp("Sarthe/roads.shp", "Sarthe/roads.dbf", "route_Sarthe/roads_{}.shp".format(type_route), type_route)

In [33]:
# Ouvre un fichier .shp et en extrait les coordonnées des extrémités de chaque route
def extract_extremities(path_shp, path_dbf):
    # Crée un dataFrame pandas vide pour stocker les coordonnées
    df = pd.DataFrame(columns=["x1", "y1", "x2", "y2"])
    # Lit le fichier .dbf et stocke les données dans un DataFrame
    df_dbf = read_dbf(path_dbf)
    # Lit le fichier .shp
    sf = shapefile.Reader(path_shp)
    # Pour chaque route
    for shape in sf.iterShapeRecords():
        # Récupère les coordonnées des extrémités de la route
        x1, y1, x2, y2 = shape.shape.bbox
        # Ajoute les coordonnées dans le DataFrame
        df = df.append({"x1": x1, "y1": y1, "x2": x2, "y2": y2}, ignore_index=True)
    # Retourne le DataFrame
    return df

In [34]:
df_extremities = extract_extremities("routes/roads_motorway.shp", "routes/roads_motorway.dbf")
df_extremities.head()

,x1,y1,x2,y2
0,0.173010,48.052220,0.195408,48.056787
1,0.586732,48.117485,0.688117,48.148540
2,0.756372,48.175110,0.803703,48.186148
3,0.299093,48.009573,0.300108,48.010391
4,0.581726,48.116068,0.586563,48.117637


In [41]:
# Crée un fichier chargeable dans NetLogo (format csv)
# Crée des noeuds à partir des routes (1 noeud tous les 100m)
def create_nodes(path_shp, path_dbf, path_out):
    # Lit le fichier .dbf et stocke les données dans un DataFrame
    df_dbf = read_dbf(path_dbf)
    # Lit le fichier .shp
    sf = shapefile.Reader(path_shp)
    # Crée un nouveau fichier .csv
    with open(path_out, "w") as file:
        # Ecrit les entêtes
        file.write("id,x,y\n")
        # Pour chaque route
        for shape in sf.iterShapeRecords():
            # Récupère les coordonnées des extrémités de la route
            x1, y1, x2, y2 = shape.shape.bbox
            # Calcule la distance entre les deux extrémités
            distance = ((x2 - x1)**2 + (y2 - y1)**2)**0.5
            # Calcule le nombre de noeuds à créer
            nb_nodes = int(distance / 100) + 1
            # Calcule le pas entre chaque noeud
            step_x = (x2 - x1) / nb_nodes
            step_y = (y2 - y1) / nb_nodes
            # Pour chaque noeud
            for i in range(nb_nodes):
                # Calcule les coordonnées du noeud
                x = x1 + i * step_x
                y = y1 + i * step_y
                # Ecrit le noeud dans le fichier .csv
                file.write("{},{}\n".format(x, y))

In [42]:
create_nodes("routes/roads_motorway.shp", "routes/roads_motorway.dbf", "routes/nodes_motorway.csv")

In [6]:
# Stocke dans un DataFrame les informations shp de la première route de type "motorway"
sf = shapefile.Reader("routes/roads_motorway.shp")
shape = sf.shapeRecords()[0]
# Affiche les informations shp de la première route de type "motorway"
shape.shape.__dict__

{'shapeType': 3,
 'points': [(0.1730102, 48.0522196),
  (0.1745596, 48.0525898),
  (0.1770621, 48.0531295),
  (0.1819828, 48.0541136),
  (0.183779, 48.0544692),
  (0.1854823, 48.0548091),
  (0.1913287, 48.0559769),
  (0.1954081, 48.0567868)],
 'parts': [0],
 '_errors': {},
 '_Shape__oid': 0,
 'bbox': [0.1730102, 48.0522196, 0.1954081, 48.0567868]}

In [7]:
# Crée un fichier .shp contenant les points de chaque route d'un type spécifié
def create_points(path_shp, path_out):
    # Lit le fichier .shp
    sf = shapefile.Reader(path_shp)
    # Crée un nouveau fichier .shp
    w = shapefile.Writer(path_out, shapefile.POINT)
    # Copie les attributs du fichier .shp
    w.fields = list(sf.fields)
    # Pour chaque route
    for shape in sf.iterShapeRecords():
        # Pour chaque point de la route
        for point in shape.shape.points:
            # Copie le point dans le nouveau fichier .shp
            w.point(*point)
            w.record(*shape.record)
    w.close()

In [8]:
create_points("routes/roads_motorway.shp", "routes/points_motorway.shp")

In [9]:
# Compte le nombre de points dans un fichier .shp
def count_points(path_shp):
    # Lit le fichier .shp
    sf = shapefile.Reader(path_shp)
    # Compte le nombre de points
    nb_points = 0
    for shape in sf.iterShapeRecords():
        nb_points += len(shape.shape.points)
    # Retourne le nombre de points
    return nb_points

count_points("routes/points_motorway.shp")

3195

In [10]:
create_points("routes/roads_primary.shp", "routes/points_primary.shp")
count_points("routes/points_primary.shp")

13609